
follow on to https://www.philipzucker.com/overload_bool/

Crosshair. Can I symexec an interpreter (riscv?) using crosshair?
Or via my thing

CF mentioned JIT is doing same thing. Build a toy jit on that principle?
output wasm maybe? use wasmtime?


fork based nondeterminsims
fork based continuations
https://discuss.systems/@pkhuong/113715738939174491
fork based continuations to implement closures. Need some kind of pipe mechanism
https://mainisusuallyafunction.blogspot.com/2012/02/continuations-in-c-with-fork.html
https://web.archive.org/web/20071101050321/http://homepage.mac.com/sigfpe/Computing/continuations.html old sigfpe site?
https://github.com/trishume/telefork

https://github.com/kmcallister/forkallcc
https://archive.jlongster.com/Whats-in-a-Continuation
https://news.ycombinator.com/item?id=42510696 into cps never to return

cps and destination passing style. Hmm.

https://stackoverflow.com/questions/48526687/call-cc-in-python-possible
https://github.com/python-greenlet/greenlet
https://gist.github.com/apg/1151419 revised callcc
https://wiki.c2.com/?StacklessPython https://en.wikipedia.org/wiki/Stackless_Python
https://doc.pypy.org/en/latest/stackless.html pypy implements this?
https://pavpanchekha.com/esp/continuations.html
http://blog.sigfpe.com/2007/02/exceptions-disjunctions-and.html

Count number of usages. Linearity garbage collection. Local garbage collection in C.

http://www.popcornlinux.org/ 
https://news.ycombinator.com/item?id=40751468 CRIU, a project to implement checkpoint/restore functionality for Linux.  https://criu.org/Main_Page checkpoint restore in userspace
 
https://en.wikipedia.org/wiki/Express_Data_Path XDP

microvms firecracker kata containers
https://github.com/infracloudio/awesome-microvm
https://www.qemu.org/docs/master/system/i386/microvm.html

unikernels

Data plane acceleration

https://www.cl.cam.ac.uk/teaching/2324/R277/handout-delimited-continuations.pdf jeremy yallop note on delimited continuation
https://news.ycombinator.com/item?id=28287312 all you need is call/cc . Suggestion that effect handlers are the more mdern thin
"You can implement delimited continuations with regular call/cc and a mutable cell. Of course, like all abstractions based on raw call/cc, it's not modular: you can't do that and use the result inside, for example, a multitasking scheduler that itself uses raw call/cc as well.
http://okmij.org/ftp/continuations/against-callcc.html#traps
Closer to your point, there are a few other implementations that offer delimited continuations natively. See https://wingolog.org/archives/2010/02/26/guile-and-delimited... for a 11 year old post on adding them to guile scheme."
https://news.ycombinator.com/threads?id=pkhuong


Could use my simple symexec to symexec an interpreter.

buildit

What does rosette offer really?

Klee

Actually reifying

Making a version that works in C++ or Rust as library.




# Reflection

If I kept a strucutred path tree could be nice. Would need to keep maybe some cotinuation. Blech

We could choose to not record conditions.


In [37]:
import random
from z3 import *
def symexec(*vs, limit=100):
    def wrapper(f):
        trace = []
        traces = []
        # a shared solver being pushed and popped is probably more efficient
        s = Solver()
        def branch(self):
            # branch gets called on every branch (every time a z3 expression is asked to be converted to a concrete bool)
            s.push()
            s.add(And([ cond == take for (take, cond) in trace])) # TODO: I could move push pop around to avoid this full push
            s.add(self)
            res = s.check()
            s.pop()
            if res == sat:
                trace.append((True, self))
                return True
            else:
                # it was not possible to take the branch
                trace.append((False, self)) # keep tuple of (cond, take)?
                return False
        BoolRef.__bool__ = branch # monkey patch in the __bool__ overload
        for i in range(limit):
            if s.check() == unsat: # If no more branches possible, stop
                break
            trace = [] # reset the trace
            res = f(*vs) # run the function
            # res = z3.simplify(res) # might be nice.
            if res is None:
                del BoolRef.__bool__
                raise ValueError("Function returns None on path", trace)
            traces.append((trace,res)) # record the result of the run
            s.add(Not(And([ cond == take for (take, cond) in trace]))) # disallow exact trace again
        del BoolRef.__bool__
        return traces
    return wrapper




@symexec(z3.Int("x"))
def foo(x):
    if x > 3:
        if x == 4:
            return x
        else:
            return x - 2
    else:
        return x*3
    
foo


[([(True, x > 3), (True, x == 4)], x),
 ([(True, x > 3), (False, x == 4)], x - 2),
 ([(False, x > 3)], x*3)]

In [45]:
def compile(traces):
    print(traces)
    if len(traces) == 1:
        trace, res = traces[0]
        assert len(trace) == 0
        return res
    cond = traces[0][0][0][1]
    print(cond)
    assert all(trace[0][0][1].eq(cond) for trace in traces)
    trues =  [(trace[1:],res) for trace, res in traces if trace[0][0]]
    falses = [(trace[1:],res) for trace, res in traces if not trace[0][0]]
    return z3.If(cond, compile(trues), compile(falses))
compile(foo)

[([(True, x > 3), (True, x == 4)], x), ([(True, x > 3), (False, x == 4)], x - 2), ([(False, x > 3)], x*3)]
x > 3
[([(True, x == 4)], x), ([(False, x == 4)], x - 2)]
x == 4
[([], x)]
[([], x - 2)]
[([], x*3)]


If(x > 3, If(x == 4, x, x - 2), x*3)

In [7]:
@symexec(z3.Int("x"))
def foo(x):
    if x == 3:
        return x
    elif x == 4:
        return x + 1
    else:
        return x - 1
    
foo

[([x == 3], x),
 ([Not(x == 3), x == 4], x + 1),
 ([Not(x == 3), Not(x == 4)], x - 1)]

continuation is manually captured in the todo_trace


In [ ]:
from z3 import *
def symexec(*vs, limit=100):
    def wrapper(f):
        trace = []
        traces = []
        todo = [[]]
        todo_trace = None
        # a shared solver being pushed and popped is probably more efficient
        s = Solver()
        def branch(self):
            # branch gets called on every branch (every time a z3 expression is asked to be converted to a concrete bool)
            if len(todo_trace) != 0:
                take = todo_trace.pop()
                trace.add(take)

            res = s.check()
            s.pop()
            if res == sat:
                trace.append(self)
                return True
            else:
                # it was not possible to take the branch
                trace.append(z3.Not(self)) # keep tuple of (cond, take)?
                return False
        BoolRef.__bool__ = branch # monkey patch in the __bool__ overload
        for i in range(limit):
            if s.check() == unsat: # If no more branches possible, stop
                break
            trace = [] # reset the trace
            res = f(*vs) # run the function
            # res = z3.simplify(res) # might be nice.
            if res is None:
                BoolRef.__bool__ = None
                raise ValueError("Function returns None on path", trace)
            traces.append((trace,res)) # record the result of the run
        BoolRef.__bool__ = None
        return traces
    return wrapper




@symexec(z3.Int("x"))
def foo(x):
    if x > 3:
        if x == 4:
            return x
        else:
            return x - 2
    else:
        return x*3
    
foo

In [4]:
def traces_to_fn(traces, sort):
    if len(traces) == 0:
        return z3.FreshConst(sort, prefix="undefined")
    pathcond, res = traces[0]
    return z3.If(z3.And( pathcond), res, traces_to_fn(traces[1:], sort))
traces_to_fn(foo, z3.IntSort())

If(And(x > 3, x == 4),
   x,
   If(And(x > 3, Not(x == 4)),
      x - 2,
      If(And(Not(x > 3)), x*3, undefined!0)))

In [ ]:
def simplify_traces(traces):
    for pathcond, res in traces:

    return z3.If(z3.And( pathcond), res, traces_to_fn(traces[1:], sort))
traces_to_fn(foo, z3.IntSort())

In [ ]:
from typing import NamedTuple
from dataclasses import dataclass

@dataclass
class Trie():
    cond : z3.BoolRef
    true : 'Trie'
    false : 'Trie'

def insert(trie, pathcond, res):
    if len(pathcond) == 0:
        return res

    for cond in pathcond:
        if trie is None:
            trie = Trie(cond, None, None)

    if cond in trie:
        return Trie(cond, insert(trie[cond], rest, res), trie[cond].false)
    else:
        return Trie(cond, insert({}, rest, res), insert({}, rest, res))

def trie_of_traces(traces):
    z3.is_not(p)



In [ ]:
im

In [36]:


def lookup_trie(trie, key):
    node = trie
    for k in key:
        #k = (k.get_id(), k)
        node = node.get(k)
        if node is None:
            return None
    return node

def store_trie(trie, key, value):
    node = trie
    for k in key[:-1]:
        #k = (k.get_id(), k)
        print(k)
        if k not in node:
            node[k] = {}
        node = node[k]
    node[key[-1]] = value

def trace_trie(traces):
    trie = {}
    for trace,res in traces:
        print(trie, trace, res)
        store_trie(trie, trace, res)
    return trie

def compress_trie(traces):
    # If only one path, compress away that node
    # If all paths have same result, compress away.
    # probably best to do bottom up
    pass

t = {}
#store_trie(t, "abc" ,3 )
#print(foo)
#del BoolRef.__bool__
ctrie = trace_trie(foo)

def compile(ctrie):
    if isinstance(ctrie,dict):
        assert len(ctrie) == 2
        cond = [c for c in ctrie.items() ]
        z3.If()
    else:
        return ctrie

#t

{} [x == 3] x
{x == 3: x} [Not(x == 3), x == 4] x + 1
Not(x == 3)
{x == 3: x, Not(x == 3): {x == 4: x + 1}} [Not(x == 3), Not(x == 4)] x - 1
Not(x == 3)


{x == 3: x, Not(x == 3): {x == 4: x + 1, Not(x == 4): x - 1}}

In [21]:
x = z3.Int("x")
d = {x  == 3: None}
(x == 3) in d

TypeError: 'NoneType' object is not callable

# Buildit

# Cross Hair
Cross uses audit hooks to save you from some possible side effects. That's interesting

Hmm. This is really not setup or documented to be a library

Nah. This is not what I want.


In [4]:
from crosshair.core_and_libs import analyze_function

def toy_function(x: int) -> int:
    if x > 0:
        return x + 1
    elif x < 0:
        return x - 1
    else:
        return 0

analyze_function(toy_function)

[]

In [ ]:
import crosshair.core as core
core.analyze_function(toy_function)


TypeError: explore_paths() missing 2 required positional arguments: 'options' and 'search_root'

In [1]:
import crosshair
dir(crosshair)

['IgnoreAttempt',
 'NoTracing',
 'ResumedTracing',
 'StateSpace',
 'SymbolicFactory',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__license__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__status__',
 '__version__',
 'auditwall',
 'codeconfig',
 'condition_parser',
 'copyext',
 'core',
 'debug',
 'deep_realize',
 'dynamic_typing',
 'enforce',
 'env_info',
 'fnutil',
 'options',
 'patch_to_return',
 'realize',
 'register_contract',
 'register_patch',
 'register_type',
 'smtlib',
 'statespace',
 'stubs_parser',
 'sys',
 'tracers',
 'type_repo',
 'util',
 'with_realized_args',
 'z3util']

# Klee
Alastair was trying to get Klee on Rust.
Klee is in llvm 13? that's insane.



In [ ]:
 docker pull klee/klee
 docker run --rm -ti --ulimit='stack=-1:-1' klee/klee